In [1]:
# 1. Library imports
import pandas as pd
from IPython.display import Markdown, display
from extraction import llmAgent
from ppi_deprescribe import merge_results, ppi_deprescribe
import os 
from pathlib import Path 


f:\LangChain\DataSci210_MedicationDeprescriber\src\extraction.py:11: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  embeddings = HuggingFaceEmbeddings(model_name="NeuML/pubmedbert-base-embeddings")
f:\LangChain\.venv\lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [2]:

path = Path('F:/LangChain/data')
groq = os.environ['groqkey']
pd.set_option('display.max_colwidth', None)

In [3]:

def deprescribe(key):
    # extract information
    llm_agent = llmAgent(groq_key=groq, data_path=path)

    results_dict = {
        "diagnosis_dict": llm_agent.extract_diagnosis(encounter_key=key),
        "encounter_dict": llm_agent.extract_encounter_info(encounter_key=key),
        # Is the reasoning in the json or sepearte?
        # Should the reasoning be included in any of them or just the diangosis with the reasoning seperate?
        "notes_dict": llm_agent.extract_notes(encounter_key=key),
    }
    print(results_dict['notes_dict'])
    # # #   master formatter step   # # #
    # merge the diagnosis booleans (just use OR logic for now)
    # make a final "reasoning" behind the recommendation
    final_dict = merge_results(results_dict=results_dict)

    # feed the three reasonings to LLM to get a single summary
    final_reasoning = llm_agent.summarize_reasonings(results_dict=results_dict)

    # # #   get recommendation from PPI algorithm   # # #
    recommendation_str = ppi_deprescribe(patient_diagnosis=final_dict)
    return recommendation_str, final_reasoning
    

In [4]:
labeled_data = pd.read_csv('F:/LangChain/data/LabeledResponses.csv')
#labeled_data = labeled_data.head(1)
labeled_data.head()

,key,reason,recommendation,GS_response,Note
0,D6253A5CE371EA,example with gastrointestinal bleed should continue PPI,continue,"This patient has a history of a gastrointestinal bleeding which was present on the admission diagnosis. The pantoprazole 20mg taken twice daily should be continued. The GI bleed is mentioned in the note from 09/14/2023, on 09/17/2023, and on 09/20/2023.",NaN
1,DFAFED1811B871,example with a documented history of a GI bleed should continue PPI,continue,This patient has a history of a gastrointestinal bleed in the present on admit diagnosis. The lansoprazole 30mg daily should be continued. Note from 09/02/2023 and 09/03/2023 mention the GI bleed.,NaN
2,D4865B8BBB294E,example with GI bleeding that occurred during inpatient stay should continue PPI,continue,GI bleeding developed during hospital stay and is present in the hospital acquired diagnosis. The patient should continue the omeprazole 20mg taking once daily. 09/13/2023 note mentions bleeding ulcer.,NaN
3,D00F3A8D5F43B2,example with symptoms of heartburn (upper GI symptoms) evaluate for endoscopy to determine if PPI is needed,deprescribe,09/20/2023 inpatient nursing note mentions use of pantoprazole for heartburn. Attempt deprescribing of lansoprazole 30mg at discharge.,NaN
4,D8017C77BA15FA,"14d h pylori treatment if pt treated for 2wks and asymptomatic, stop PPI",stop,Stop pantoprazole 40mg when H. Pylori treatment scheduled from 07/24-08/07 is complete. Documented in note from Anesthesiology on 09/01/2023,NaN


In [5]:
def apply_deprescribe(row):
    try:
        recommendation_str, final_reasoning = deprescribe(row['key'])
        return pd.Series([recommendation_str, final_reasoning])
    except:
        return pd.Series([None, None])

labeled_data[['Rec_Returned', 'Reasoning']] = labeled_data.apply(apply_deprescribe, axis=1)

{'Mild to moderate esophagitis': False, 'GERD': False, 'Peptic Ulcer Disease': False, 'Upper GI symptoms': True, 'ICU Stress Ulcer Prophylaxis': False, 'Barretts Esophagus': False, 'Chronic NSAID use with bleeding risk': False, 'Severe esophagitis': False, 'Documented history of bleeding GI ulcer': True, 'Reasoning': "The patient has a history of gastrointestinal bleeding and has been diagnosed with anemia. The patient's recent endoscopy and colonoscopy results show no active bleeding, but the patient's history of bleeding and anemia suggests a high risk for future bleeding. The patient's medications, including anticoagulants and antiplatelets, increase the risk of bleeding. The patient's condition requires close monitoring and management to prevent future bleeding."}
{'Mild to moderate esophagitis': False, 'GERD': False, 'Peptic Ulcer Disease': True, 'Upper GI symptoms': True, 'ICU Stress Ulcer Prophylaxis': False, 'Barretts Esophagus': False, 'Chronic NSAID use with bleeding risk': F

In [6]:
labeled_data.head()

,key,reason,recommendation,GS_response,Note,Rec_Returned,Reasoning
0,D6253A5CE371EA,example with gastrointestinal bleed should continue PPI,continue,"This patient has a history of a gastrointestinal bleeding which was present on the admission diagnosis. The pantoprazole 20mg taken twice daily should be continued. The GI bleed is mentioned in the note from 09/14/2023, on 09/17/2023, and on 09/20/2023.",NaN,continue,"The patient has a documented history of gastrointestinal bleeding, which is consistent with upper GI symptoms and a history of bleeding in the gastrointestinal tract. The primary diagnosis is gastrointestinal hemorrhage, unspecified, and the patient has been diagnosed with anemia, indicating a high risk for future bleeding. The patient's medications, including anticoagulants and antiplatelets, increase the risk of bleeding."
1,DFAFED1811B871,example with a documented history of a GI bleed should continue PPI,continue,This patient has a history of a gastrointestinal bleed in the present on admit diagnosis. The lansoprazole 30mg daily should be continued. Note from 09/02/2023 and 09/03/2023 mention the GI bleed.,NaN,continue,"The patient's history and current conditions are evaluated for gastrointestinal issues. \n\nIn one case, the patient has a documented history of bleeding GI ulcer, but lacks information on current GI symptoms or other conditions. \n\nIn another case, there is no mention of gastrointestinal issues, with the primary diagnosis being sepsis.\n\nIn the third case, the patient has a history of peptic ulcer disease, is experiencing upper GI symptoms, and has a documented history of bleeding GI ulcer, with current medications and lab results indicating possible related issues."
2,D4865B8BBB294E,example with GI bleeding that occurred during inpatient stay should continue PPI,continue,GI bleeding developed during hospital stay and is present in the hospital acquired diagnosis. The patient should continue the omeprazole 20mg taking once daily. 09/13/2023 note mentions bleeding ulcer.,NaN,continue,"The reasoning provided in the given JSON files can be summarized as follows:\n\n- The patient's medical history and current symptoms were evaluated to determine the presence of gastrointestinal (GI) conditions. \n- In one case, the patient had a documented history of GI bleeding, suggesting a history of bleeding GI ulcer, but there was no information about other conditions.\n- In another case, there was no information about the patient's GI health, making it impossible to determine the presence of GI conditions.\n- In the third case, the patient had a history of GERD, peptic ulcer disease, and upper GI symptoms, and had experienced episodes of melena requiring transfusion, indicating a high risk for upper GI bleeding and supporting the assessment of a documented history of bleeding GI ulcer."
3,D00F3A8D5F43B2,example with symptoms of heartburn (upper GI symptoms) evaluate for endoscopy to determine if PPI is needed,deprescribe,09/20/2023 inpatient nursing note mentions use of pantoprazole for heartburn. Attempt deprescribing of lansoprazole 30mg at discharge.,NaN,deprescribe,"The provided JSON information does not contain specific details about the patient's medical conditions that relate to the listed gastrointestinal conditions. However, in one instance, the patient has been experiencing upper GI symptoms, but these symptoms are more likely related to a retroperitoneal mass and possible bowel obstruction rather than the listed conditions."
4,D8017C77BA15FA,"14d h pylori treatment if pt treated for 2wks and asymptomatic, stop PPI",stop,Stop pantoprazole 40mg when H. Pylori treatment scheduled from 07/24-08/07 is complete. Documented in note from Anesthesiology on 09/01/2023,NaN,continue,"The reasoning provided in the three JSON files can be summarized as follows:\n\nThe patient's medical conditions were evaluated based on the provided information. In the first two cases, the information was insufficient or unrela

In [7]:
labeled_data.to_csv('F:/LangChain/data/LabeledResponses_ReturnedResponse.csv', index=False)